### Change Log
---
Integrating custom vectorizer, end to end integration
Overall pipeline works, and need to train fasttext

In [1]:
import sys
sys.path.append('/data/extensibleFramework/')

In [2]:
from extensibleFramework.src.models import convolution_neural_network
from extensibleFramework.src.models import recurrent_nn_with_attention
from extensibleFramework.src.models import feed_forward_network
from extensibleFramework.src.models import extra_layers
from extensibleFramework.src.utils import saving_and_loading
from extensibleFramework.src.utils import grid_search
from extensibleFramework.src.models import config
from extensibleFramework.src.utils import biomedical_tokenizer
from torchtext import vocab
from torchtext import data
import pandas as pd
import chakin
import matplotlib.pyplot as plt
from torchtext import data
import nltk
import json
from torchtext import vocab
from tqdm import tqdm
import torch
import random
import re
import pandas as pd
import os
import numpy as np
import sys
import random
import tarfile
import urllib
from torchtext import data
import datetime
from extensibleFramework.src.utils import custom_vectorizer
from nltk.tokenize import MWETokenizer
import json
SAL = saving_and_loading.objectManager()
tokenizer = biomedical_tokenizer.getToknizer()
from nltk.tokenize import MWETokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CV = custom_vectorizer.Vectorizer('/data/extensibleFramework/extensibleFramework/embedidngs/fastText.model')


Loading Model
Loaded Model Successfully


In [3]:
def tokenize(sentiments):
    return sentiments.lower().split(" ")
def to_categorical(x):
    x = int(x)
    if x == 1:
        return [0,1]
    if x == 0:
        return [1,0]

In [4]:
REVIEW = data.Field(sequential=True , tokenize=tokenize, use_vocab = True, lower=True,batch_first=True)
LABEL = data.Field(is_target=True,use_vocab = False, sequential=False, preprocessing = to_categorical)
fields = {'review': ('review', REVIEW), 'label': ('label', LABEL)}
train_data , test_data = data.TabularDataset.splits(
                            path = '',
                            train = '/data/extensibleFramework/extensibleFramework/data/processed/train.json',
                            test = '/data/extensibleFramework/extensibleFramework//data/processed/test.json',
                            format = 'json',
                            fields = fields)

In [5]:
REVIEW.build_vocab(train_data, test_data, max_size=400000)

In [6]:
CV.prepare_vectors(REVIEW.vocab.itos,"/data/extensibleFramework/extensibleFramework/embedidngs/Custom_vectors.txt")

/data/extensibleFramework/extensibleFramework/src/utils/custom_vectorizer.py:29: UserWarning: ALREADY EXSTS : /data/extensibleFramework/extensibleFramework/embedidngs/Custom_vectors.txt   make sure you dont over write previous vectors
  warnings.warn("ALREADY EXSTS : "+ destination_file +"   make sure you dont over write previous vectors")


Go ahead and overwrite (y/n)n


False

In [7]:
vec = vocab.Vectors(name = "../embedidngs/glove.6B.100d.txt",cache = "./")

In [8]:
REVIEW.build_vocab(train_data, test_data, max_size=400000, vectors=vec)

In [9]:
sentiment_vocab = REVIEW.vocab

In [10]:
sentiment_vocab.vectors

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3456,  0.0004, -0.1884,  ...,  0.3707, -0.1417, -0.7469],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [11]:
SEED = 1
split = 0.80
batch_size  =32

In [12]:
class parameters:
    def __init__(self):
        self.cnn_rnn_vocab_size = len(sentiment_vocab)
        self.cnn_rnn_embed_dim = 100
        self.cnn_rnn_class_num = [200]
        self.cnn_out_channel_num = [24]
        self.cnn_kernel_sizes =  [3,4,5]
        self.rnn_n_layers = [1,2,3]
        self.rnn_hidden_size =  [128]
        self.use_pretrained_weights = True
        self.cnn_rnn_weights =  sentiment_vocab.vectors
        self.cnn_rnn_weight_is_trainable =   False
        self.dropout = [0.2]
        self.batch_size = batch_size
        self.merge_mode = "CONCAT"
        self.ffn_activation =  "Relu"
        self.ffn_final_output_classes =  2
        self.ffn_perceptron_per_layer = [[100,50, 25]]
        self.ffn_layer_wise_dropout = 0.2
        self.learning_rate =  [0.2]
        self.momentum = [0.9]
        self.device = device

In [13]:
SP = grid_search.searchParameters(parameters)


In [14]:
def tokenize(sentiments):
    return tokenizer.tokenizer.tokenize(sentiments.split())

def to_categorical(x):
    x = int(x)
    if x == 1:
        return [0,1]
    if x == 0:
        return [1,0]

In [15]:
## Tokenize test
tokenize("i am like acne Vulgaris the blah sleep disorder .")

['i', 'am', 'like', 'acne', 'Vulgaris', 'the', 'blah', 'sleep_disorder', '.']

In [16]:
train_iter, test_iter = data.Iterator.splits(
        (train_data, test_data), sort_key=lambda x: len(x.review),
        batch_sizes=(batch_size,batch_size), device=device)

In [17]:
for batch in test_iter:
    feature, target = batch.review, batch.label
    print(feature.data.shape, target.data.shape)
    break

torch.Size([32, 40]) torch.Size([32, 2])


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class ensemble_model(nn.Module):
    def __init__(self, config_object):
        super(ensemble_model, self).__init__() 
        self.cnn = convolution_neural_network.cnn_text(config_object)
        self.rnn_attention = recurrent_nn_with_attention.RNNAttentionModel(config_object)
        self.merge_layer = extra_layers.MergeAndFlattern(config_object)
        self.ffn = feed_forward_network.ffn(config_object)
    def forward(self, x):
        cnn_output = self.cnn(x)
        rnnAttention_output = self.rnn_attention(x)
        merge_layer_output = self.merge_layer(cnn_output,rnnAttention_output )
        final_output = self.ffn(merge_layer_output)
        
#         print("cnn_output : ",cnn_output.shape)
#         print("rnn_input : ",rnnAttention_output.shape)
#         print("merge_layer_output : ", merge_layer_output.shape)
#         print("final_output : ", final_output.shape)
        return final_output

In [19]:
for parma_set in SP.random_search(1):
    configuration = config.Config(parma_set)


In [20]:
cnn_input = torch.Tensor(np.random.randint(5, size=[8, 10])).long().to(device)
rnn_input = torch.Tensor(np.random.random([8, 10])).long().to(device) #batch_size, input_size

In [21]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    rounded_preds = torch.argmax(preds, dim=1)
    correct = (rounded_preds == torch.argmax(y, dim=1)).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [22]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        feature, target = batch.review, batch.label
        optimizer.zero_grad()
        predictions = model(feature.to(device))            
        loss = criterion(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
        loss.backward()
        optimizer.step()
        acc = binary_accuracy(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return model, epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
def test_accuracy_calculator(model, test_iterator):
    epoch_acc = 0
    for batch in test_iterator:
        if batch.review.shape[0] ==  32:
            feature, target = batch.review, batch.label
            predictions = model(feature.to(device))            
            acc = binary_accuracy(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
            epoch_acc += acc.item()
    return  epoch_acc / len(test_iterator)

In [24]:
epochs  = 100
log_interval = 1
loss = []
accuracy = []
test_accuracy = []
for parma_set in SP.random_search(1):
    configuration = config.Config(parma_set)
    EM  = ensemble_model(configuration)
    EM  = EM.to(device)
    optimizer = torch.optim.SGD(EM.parameters(), lr=0.1,momentum=0.9)
    criterion = nn.MSELoss()
    criterion = criterion.to(device)
    
    for i in tqdm(range(epochs)):
        if (i != 0 and i%10 == 0 ):
            for param_group in optimizer.param_groups:
                param_group['lr'] = param_group['lr']/2
            print(" === New Learning rate : ", param_group['lr'], " === ")

        model, epoch_loss, epoch_acc = train(EM, train_iter, optimizer, criterion)

        test_acc = test_accuracy_calculator(model, test_iter)
        accuracy.append(epoch_acc)
        loss.append(epoch_loss)
        test_accuracy.append(test_acc)
        print(epoch_acc,test_acc,epoch_loss)
    del configuration.device
    detailed_params  = json.dumps(configuration, default=lambda x: x.__dict__)
    SAL.saver(EM, "train",model_performance_metrics={"Train_accuracy":epoch_acc,"Test accuracy": test_acc, "Epoch Loss":epoch_loss}, detailed_params=detailed_params )

  1%|          | 1/100 [00:39<1:05:14, 39.54s/it]

0.4982 0.49880573248407645 0.25117238309383394


  2%|▏         | 2/100 [01:14<1:02:24, 38.21s/it]

0.50345 0.49880573248407645 0.2511788880109787


  3%|▎         | 3/100 [01:48<59:34, 36.85s/it]  

...

0.9858 0.8984872611464968 0.013766913559870364


 99%|█████████▉| 99/100 [55:16<00:32, 32.43s/it]

0.98605 0.8984872611464968 0.013495832858053268


100%|██████████| 100/100 [55:49<00:00, 32.38s/it]

0.986 0.8990843949044586 0.013646213521917562


In [32]:
parma_set= next(SP.random_search(1))


In [33]:
parma_set.batch_size

32